### SQuAD 데이터 입출력

SQuAD 데이터셋에 대한 모델 입출력은 아래와 같다.

`question`과 `text`를 구분자(`[CLS]`, `[SEP]`)로 구분하여 아래와 같이 `inputs`을 만든다

위 입력에 대한 정답인 `start_position`과 `end_position`을 인덱스로 나타낸다.

`inputs`의 10번째 토큰부터 12번째 토큰은 `['a', 'nice', 'puppet']`이며 `"Who was Jim Hensom"`에 대한 정답이다.

### Dataset 및 DataLoader 생성

In [1]:
import torch
from tqdm import tqdm, trange
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from torch import nn
from torch.utils.data import DataLoader
from dataloader import SquadDataLoader
from dataset import SquadDataset

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [3]:
train_dataset = SquadDataset('data', tokenizer, is_train=True)

cache file does not exist
examples: 130319


100%|██████████| 130319/130319 [08:18<00:00, 261.28it/s]


is_training: True


In [4]:
train_dataloader = SquadDataLoader(train_dataset, batch_size=16, is_inference=False, shuffle=True)

In [5]:
for i, batch in enumerate(train_dataloader):
    if i > 10:
        break
    input_ids, input_mask, segment_ids, cls_index, p_mask, start_positions, end_positions = batch
    print(input_ids.shape, input_mask.shape, segment_ids.shape, cls_index.shape, p_mask.shape, start_positions.shape, end_positions.shape)

torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16])
torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16])
torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16])
torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16])
torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16])
torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16])
torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16, 384]) torch.Size([16]) torch.Size([16])
torch.Size([16, 384]) torch

### Train

In [6]:
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased').cuda()
model.train

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

<bound method Module.train of BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [7]:
# Optimizer와 Loss 함수는 가장 일반적인 것으로 정의했다.
# 이 노트북 파일의 목적은 BERT를 이용해서 높은 성능의 모델을 간편하게 만들 수 있다는 것을 보여주기 위함이다.
# Optimizer와 Loss를 최적화할 경우 좋은 성능이 나온 이유를 잘 설명할 수 없다.
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)
loss = nn.CrossEntropyLoss()

/home/jkfirst/workspace/git/Pytorch_Retinaface/env_ret/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [8]:
n_epoch = 10

In [9]:
def train(model, dataloader, optimizer):
    tbar = tqdm(dataloader, desc='Training', leave=True)
    
    total_loss = 0.0
    for i, batch in enumerate(tbar):
        optimizer.zero_grad()
        
        # cls_index와 p_mask는 XLNet 모델에 사용되므로 BERT에서는 사용하지 않는다.
        input_ids, input_mask, segment_ids, cls_index, p_mask, start_positions, end_positions = batch
        
        # to cuda
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()
        segment_ids = segment_ids.cuda()
        start_positions = start_positions.cuda()
        end_positions = end_positions.cuda()
        
        # train model
        #out = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        inputs = {
            'input_ids': input_ids,
            'token_type_ids': segment_ids,
            'attention_mask': input_mask,
        }
        out = model(**inputs, start_positions=start_positions, end_positions=end_positions)
        loss = out.loss

        loss.backward()
        optimizer.step()
        
        total_loss += loss.data.item()
        tbar.set_description("Average Loss = {:.4f})".format(total_loss/(i+1)))
        

In [10]:
for i in range(n_epoch):
    train(model, train_dataloader, optimizer)

Average Loss = 0.0503): 100%|██████████| 8247/8247 [49:38<00:00,  2.77it/s]


In [11]:
torch.save(model.state_dict(), 'squad_model.bin')

In [12]:
! ls -alh *.bin

-rw-rw-r-- 1 jkfirst jkfirst 416M  8월  2 13:10 squad_model.bin


### Inference